In [78]:
import csv
from rdflib import Graph, URIRef, Literal, Namespace, RDF, RDFS, XSD
from datetime import datetime
from rdflib.plugins.sparql import prepareQuery

# Créez un espace de nom pour votre ontologie
kesaio = Namespace("http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/")
common_core_ns = Namespace("http://www.ontologyrepository.com/CommonCoreOntologies/")

In [79]:
def sévérité(kesaio, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, seisme_uri, nombre_morts):
    if nombre_morts in severite_niveau_0:
        severite = "Niveau 0"
    elif nombre_morts in severite_niveau_1:
        severite = "Niveau 1"
    elif nombre_morts in severite_niveau_2:
        severite = "Niveau 2"
    else:
        severite = "Niveau inconnu"

        # Ajoutez une propriété de sévérité à l'instance de catastrophe
    g.add((seisme_uri, kesaio["sévérite_de_la_catastrophe"], Literal(severite, datatype=XSD.string)))

In [80]:
# Ouvrez le fichier CSV contenant les données
def creer_victimes(kesaio, common_core_ns, g, dcd_uri, bls_uri, row, seisme_uri):
    for i in range(0, int(row["nombreMorts"])):  # Créez 25 victimes
            # Créez une instance unique de la classe "Etat_victime"
        etat_victime_id = kesaio["Etat_victime_Mort" + str(i)]  # Par exemple, "Etat_victime_1", "Etat_victime_2", ...
            
            # Utilisez l'instance "DCD" pour attribuer l'état de décédé à chaque instance d'état de victime
        g.add((etat_victime_id, RDF.type, kesaio["Etat_victime"]))  # Assurez-vous que la classe "Etat_victime" est définie dans votre ontologie
        g.add((etat_victime_id, common_core_ns["designated_by"], dcd_uri))  # Reliez l'état de victime à l'instance "DCD"

            # Créez une instance unique de la classe "Victime" avec un identifiant unique (Vi)
        victime_id = kesaio["Mort" + str(i)]  # Par exemple, "Vi1", "Vi2", ...
            
            # Ajoutez le triple RDF pour chaque instance de victime
        g.add((victime_id, RDF.type, kesaio["Victime"]))  # Assurez-vous que la classe "Victime" est définie dans votre ontologie
        g.add((victime_id, kesaio["has_status"], etat_victime_id))
        g.add((seisme_uri, kesaio["engendre"], victime_id))


    for i in range(0, int(row["nombreBlesse"])):  # Créez 25 victimes
            # Créez une instance unique de la classe "Etat_victime"
        etat_victime_id = kesaio["Etat_victime_Blessé" + str(i)]  # Par exemple, "Etat_victime_1", "Etat_victime_2", ...
            
            # Utilisez l'instance "DCD" pour attribuer l'état de décédé à chaque instance d'état de victime
        g.add((etat_victime_id, RDF.type, kesaio["Etat_victime"]))  # Assurez-vous que la classe "Etat_victime" est définie dans votre ontologie
        g.add((etat_victime_id, common_core_ns["designated_by"], bls_uri))  # Reliez l'état de victime à l'instance "DCD"

            # Créez une instance unique de la classe "Victime" avec un identifiant unique (Vi)
        victime_id = kesaio["Blessé" + str(i)]  # Par exemple, "Vi1", "Vi2", ...
            
            # Ajoutez le triple RDF pour chaque instance de victime
        g.add((victime_id, RDF.type, kesaio["Victime"]))  # Assurez-vous que la classe "Victime" est définie dans votre ontologie
        g.add((victime_id, kesaio["has_status"], etat_victime_id))
        g.add((seisme_uri, kesaio["engendre"], victime_id))

In [81]:
# def creer_pompiers(g, kesaio, nombre_pompiers, sapeur_pompier, common_core_ns):
#     hasRole = URIRef("http://purl.obolibrary.org/obo/RO_0000087")
#     # Créez le directeur
#     directeur_id = kesaio["Directeur"]
#     g.add((directeur_id, RDF.type, kesaio["Pompier"]))  # Assurez-vous que la classe "Pompier" est définie dans votre ontologie
#     g.add((directeur_id, common_core_ns["is_affiliated_with"], sapeur_pompier))
#     g.add((directeur_id, hasRole, kesaio["Directeur_des_secours_"]))

#     # Créez le commandant
#     commandant_id = kesaio["Commandant"]
#     g.add((commandant_id, RDF.type, kesaio["Pompier"]))  # Assurez-vous que la classe "Pompier" est définie dans votre ontologie
#     g.add((commandant_id, common_core_ns["is_affiliated_with"], sapeur_pompier))
#     g.add((commandant_id, hasRole, kesaio["Commandant_des_operations_"]))

#     # Créez le chef d'équipe
#     chef_de_equipe_id = kesaio["ChefDeEquipe"]
#     g.add((chef_de_equipe_id, RDF.type, kesaio["Pompier"]))  # Assurez-vous que la classe "Pompier" est définie dans votre ontologie
#     g.add((chef_de_equipe_id, common_core_ns["is_affiliated_with"], sapeur_pompier))
#     g.add((chef_de_equipe_id, hasRole, kesaio["Chef_dEquipe_"]))

#     # Créez les équipiers
#     for i in range(3, nombre_pompiers):  # Commencez à partir de l'index 3 pour les équipiers
#         pompier_id = kesaio["Pompier_" + str(i)]
#         g.add((pompier_id, RDF.type, kesaio["Pompier"]))  # Assurez-vous que la classe "Pompier" est définie dans votre ontologie
#         g.add((pompier_id, common_core_ns["is_affiliated_with"], sapeur_pompier))
#         g.add((pompier_id, hasRole, kesaio["Equipier_"]))

In [82]:
def enrichir(kesaio, common_core_ns, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, dcd_uri, bls_uri, creer_victimes, sévérité, row):
    date_uri = kesaio[row["date"]]
        
        # Ajoutez des triples RDF pour la classe Date
    g.add((date_uri, RDF.type, kesaio["Date"]))
        
        # Extraitz le jour, le mois et l'année du format de date "2016-02-06T00:00:00Z"
    date_str = row["date"]
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    jour = date_obj.day
    mois = date_obj.month
    annee = date_obj.year
        
        # Créez des ressources URI pour les composants jour, mois et année
    jour_uri = kesaio[f"jour_{jour}"]
    mois_uri = kesaio[f"mois_{mois}"]
    annee_uri = kesaio[f"{annee}"]

        # Ajoutez des triples RDF pour les composants jour, mois et année
    g.add((jour_uri, RDF.type, common_core_ns["GregorianDay"]))
    g.add((mois_uri, RDF.type, kesaio["Gregorian_Month"]))
    g.add((annee_uri, RDF.type, common_core_ns["GregorianYear"]))
        
        # Liez les composants jour, mois et année à l'instance de date
    g.add((date_uri, kesaio["has_day"], jour_uri))
    g.add((date_uri, kesaio["has_month"], mois_uri))
    g.add((date_uri, kesaio["has_year"], annee_uri))

    seisme_uri = kesaio[row["seismeLabel"]]
    pays_uri = kesaio[row["paysLabel"]]
    city_uri = kesaio[row["lieuLabel"]]

        
        
        # Ajoutez des triples RDF en utilisant l'espace de nom
    g.add((seisme_uri, RDF.type, kesaio["Séisme"]))
    g.add((pays_uri, RDF.type, common_core_ns["State"]))
    g.add((city_uri, RDF.type, common_core_ns["City"]))
        
        # Ajoutez des triples RDF pour les dataproperties
    g.add((seisme_uri, kesaio["nombre_de_morts"], Literal(row["nombreMorts"], datatype=XSD.integer)))
    g.add((seisme_uri, kesaio["nombre_de_blessés"], Literal(row["nombreBlesse"], datatype=XSD.integer)))
    g.add((seisme_uri, kesaio["description"], Literal(row["description"], datatype=XSD.string)))
        
        # Ajoutez des triples RDF pour les relations entre les instances
    g.add((seisme_uri, common_core_ns["occurs_at"], city_uri))
    g.add((city_uri, common_core_ns["spatial_part_of"], pays_uri))
    g.add((seisme_uri, common_core_ns["occurs_on"], date_uri))

        # Obtenez le nombre de morts à partir du CSV (assurez-vous que le champ est converti en entier)
    nombre_morts = int(row["nombreMorts"])
        
        # Déterminez la sévérité en fonction du nombre de morts
    sévérité(kesaio, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, seisme_uri, nombre_morts)

        # Créez 25 instances uniques de la classe "Victime" et leurs états correspondants
    creer_victimes(kesaio, common_core_ns, g, dcd_uri, bls_uri, row, seisme_uri)
    # Créez une instance unique de la classe "Sapeur_pompier" avec un identifiant unique (Sapeur_pompier_i)


    sapeur_pompier = kesaio["Sapeur_pompier_"]  # Par exemple, "Sapeur_pompier_0", "Sapeur_pompier_1", ...
    # Ajoutez le triple RDF pour chaque instance de sapeur pompier
    g.add((sapeur_pompier, RDF.type, kesaio["Sapeur_pompier"]))  # Assurez-vous que la classe "Sapeur_pompier" est définie dans votre ontologie
    # Utilisez la fonction pour créer 500 pompiers
    # creer_pompiers(g, kesaio, 10, sapeur_pompier, common_core_ns)


In [83]:
# Créez une instance de Graph pour votre ontologie RDF
g = Graph()

# Chargez votre ontologie RDF existante (si nécessaire)
g.parse("file8.rdf")

# Définissez les plages de nombre de morts pour chaque niveau de sévérité
severite_niveau_0 = range(0, 100)
severite_niveau_1 = range(100, 500)
severite_niveau_2 = range(500, 1000000)

dcd_uri = kesaio["DCD"]
bls_uri = kesaio["BLSS"]
disp_uri = kesaio["DSIP"]

g.add((dcd_uri, RDF.type, kesaio["victim_state_code_identifier"]))
g.add((bls_uri, RDF.type, kesaio["victim_state_code_identifier"]))
g.add((disp_uri, RDF.type, kesaio["victim_state_code_identifier"]))


with open("dataFromWikidataNew.csv", "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        # Créez une ressource URI pour chaque instance de date
        enrichir(kesaio, common_core_ns, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, dcd_uri, bls_uri, creer_victimes, sévérité, row)


# Enregistrez l'ontologie enrichie dans un fichier RDF (si nécessaire)
g.serialize("ontologie_enrichie.rdf", format="xml")

# Vous pouvez également imprimer le contenu de l'ontologie
for triple in g:
    print(triple)

(rdflib.term.BNode('N903d5273b19e4331b7da0ef813298494'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Equipe_de_Feux_de_foret'))
(rdflib.term.BNode('N897b2f9f8bcf4122a824f3dc83eaf053'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Centre_de_prémiere_intervention'))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Camion_de_citerne_feux_de_foret'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/mois_5'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/Catast

In [84]:
# What is the nature of the disaster <X>?
requete_sparql = """
SELECT ?type ?description
WHERE {
    ?seisme rdf:type ?type.
    ?seisme kesaio:description ?description .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenir le nom de la classe en extrayant la partie après le dernier "/"
    classe_nom = resultat.type.split("/")[-1]
    print("Nature:", classe_nom)
    print("Description:", resultat.description)

Nature: Séisme
Description: tremblement_de_terre_du_5_mai_2023_au_Japon,_de_magnitude_6,3


In [85]:
# When did the disaster <X> take place?
requete_sparql = """
SELECT ?jour ?mois ?annee
WHERE {
    ?seisme cco:occurs_on ?date.
    ?date kesaio:has_day ?jour .
    ?date kesaio:has_month ?mois .
    ?date kesaio:has_year ?annee .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/", 
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    jour = str(resultat.jour.split('/')[-1])
    mois = str(resultat.mois.split('/')[-1])
    annee = str(resultat.annee.split('/')[-1])

    print("Jour:", jour)
    print("Mois:", mois)
    print("Année:", annee)

Jour: jour_5
Mois: mois_5
Année: 2023


In [86]:
# Where did the disaster <X> take place?
requete_sparql = """
SELECT ?city ?pays
WHERE {
    ?seisme cco:occurs_at ?city.
    ?city cco:spatial_part_of ?pays
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    city = str(resultat.city.split('/')[-1])
    pays = str(resultat.pays.split('/')[-1])

    print("Ville:", city)
    print("Pays:", pays)

Ville: Suzu
Pays: Japon


In [87]:
# What is the criticality level of the disaster <X>?
requete_sparql = """
SELECT ?sévérité ?blessés ?morts
WHERE {
    ?seisme kesaio:nombre_de_morts ?morts .
    ?seisme kesaio:nombre_de_blessés ?blessés .
    ?seisme kesaio:sévérite_de_la_catastrophe ?sévérité .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    sévérité = str(resultat.sévérité.split('/')[-1])
    blessés = str(resultat.blessés.split('/')[-1])
    morts = str(resultat.morts.split('/')[-1])

    print("Sévérité de la catastrophe:", sévérité)
    print("Nombre de blessés:", blessés)
    print("Nombre de morts:", morts)

Sévérité de la catastrophe: Niveau 0
Nombre de blessés: 4
Nombre de morts: 2


# PLusieurs instances partagent le même rôle